In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import itertools 
import multiprocessing as mp
import numpy as np
import sys

sys.path.append('../')
from baseModels.SVM.model import SVM

In [2]:
feature_dir = '../featEngg/kernelMethods/data/featvec/'
feature_files = [f.name for f in os.scandir(feature_dir) if f.name.endswith('.csv.gz')]

In [3]:
label_file = '../data/LabelFiles/EnzymeLabelsMultiClass.csv'

In [4]:
def model_evaluate(filename,random_seed):
    df1 = pd.read_csv(feature_dir+filename,header=None)
    df2 = pd.read_csv(label_file,header=None)
    df = df1.merge(df2,on=0)
    enz_names = df[0].values
    X = df.iloc[:,1:-1].values
    y = df.iloc[:,-1].values
    if X.shape[1]<40:
        n_comp = int(0.75*X.shape[1])
    else:
        n_comp=40
    X_train, X_test, y_train, y_test,enz_train,enz_test = train_test_split(X, y,enz_names, test_size=0.25, random_state=random_seed)
    svm = SVM(X_train,X_test,y_train,y_test,verbose=False,optimize=False, pca_comp=n_comp,kern='rbf',regC=30)
    return svm.acc_train,svm.acc_test
 

In [5]:
mp.cpu_count()

8

In [6]:
pool = mp.Pool(mp.cpu_count())

In [7]:
iterable = list(itertools.product(feature_files,range(100)))

In [8]:
%%time
model_accs = list(pool.starmap(model_evaluate,iterable))

CPU times: user 90 ms, sys: 65.7 ms, total: 156 ms
Wall time: 2min 16s


In [9]:
mydict = dict()

for m,acc in zip(iterable,model_accs):
    model_name=m[0].replace('.csv.gz','')
    seed = m[1]
    if model_name not in mydict:
        mydict[model_name] = {seed:acc}
    else:
        mydict[model_name][seed] = acc

In [10]:
def get_model_stats(mname):
    train_accs = [v[0] for k,v in mydict[mname].items()]
    test_accs = [v[1] for k,v in mydict[mname].items()]
    return {'train_min':min(train_accs),'train_mean':np.mean(train_accs),
            'train_max':max(train_accs),'train_std':np.std(train_accs),
            'test_min':min(test_accs),'test_mean':np.mean(test_accs),
            'test_max':max(test_accs),'test_std':np.std(test_accs)}

In [11]:
get_model_stats('mismatchKernel')

{'train_min': 0.8604651162790697,
 'train_mean': 0.9050000000000001,
 'train_max': 0.9534883720930233,
 'train_std': 0.020143817687374305,
 'test_min': 0.4827586206896552,
 'test_mean': 0.7148275862068965,
 'test_max': 0.9655172413793104,
 'test_std': 0.0877591287838418}

In [12]:
results_dict= {mname:get_model_stats(mname) for mname in mydict.keys()}
    

In [13]:
results_df = pd.DataFrame(results_dict).T

In [14]:
results_df.to_csv('../data/SimResults/KernelResults.csv',index=True)

In [15]:
results_df.sort_values('train_mean')

,train_min,train_mean,train_max,train_std,test_min,test_mean,test_max,test_std
mismatchKernel,0.860465,0.905000,0.953488,0.020144,0.482759,0.714828,0.965517,0.087759
gappyKernel,0.848837,0.910814,0.976744,0.023375,0.448276,0.736897,0.896552,0.072930
spectrumKernel,0.860465,0.915814,0.976744,0.022911,0.448276,0.744483,0.896552,0.074772
